1. Initialize RT class
2. This contains the RT model 
3. Jan's generate method works 

In [27]:
import logging
import csv
from gt4sd.algorithms.conditional_generation.regression_transformer import  RegressionTransformer as RegressionTransformerGeneratorWrapper
from gt4sd.algorithms.conditional_generation.regression_transformer import RegressionTransformerMolecules
from rdkit import Chem
import logging
import os
import pkgutil
import tempfile
import time
from typing import List
import pandas as pd 
import fcd
import numpy as np
import uuid

from guacamol.distribution_learning_benchmark import DistributionLearningBenchmark, DistributionLearningBenchmarkResult
from guacamol.distribution_matching_generator import DistributionMatchingGenerator
from guacamol.utils.data import get_random_subset
from guacamol.utils.sampling_helpers import sample_valid_molecules
from guacamol.assess_distribution_learning import DistributionMatchingGenerator
from jointformer.models.base import BaseModel, SmilesEncoder

import os
from tqdm import tqdm
import torch
import numpy as np

from gt4sd.algorithms.conditional_generation.regression_transformer import (
    RegressionTransformer, RegressionTransformerMolecules
)


from jointformer.models.regression_transformer import _load_regression_transformer_dataset


In [ ]:
# Exemplary model implementation

from guacamol.assess_distribution_learning import DistributionMatchingGenerator
from jointformer.models.base import BaseModel, SmilesEncoder
from molecule_generation import load_model_from_directory, VaeWrapper
from molecule_generation.models.moler_vae import MoLeRVae

import os
from tqdm import tqdm
import torch
import numpy as np
from molecule_generation.utils.model_utils import load_vae_model_and_dataset
from molecule_generation.utils.moler_inference_server import _encode_from_smiles


class Moler(BaseModel, DistributionMatchingGenerator, SmilesEncoder):
    def __init__(self) -> None:
        super().__init__()
        self._model: MoLeRVae = None
        self._dataset = None
        self._tokenizer = None
        self._batch_size = None
        self._temperature = None
        self._top_k = None
        self._device = None

    def to_guacamole_generator(self, tokenizer, batch_size, temperature, top_k, device) -> DistributionMatchingGenerator:
        self._tokenizer = tokenizer
        self._batch_size = batch_size
        self._temperature = temperature
        self._top_k = top_k
        self._device = device
        return self
    
    def to_smiles_encoder(self, tokenizer, batch_size, device) -> SmilesEncoder:
        self._tokenizer = tokenizer
        self._batch_size = batch_size
        self._device = device
        return self

    def generate(self, number_samples: int):
        generated = []
        with self._model as model:
            for _ in tqdm(range(0, number_samples, self._batch_size), "Generating samples"):
                samples = model.sample(self._batch_size)
                generated.extend(samples)
        return generated[:number_samples]

    def encode(self, smiles: list[str]) -> np.ndarray:
        rets = []
        for i in tqdm(range(0, len(smiles), self._batch_size), "Encoding samples"):
            batch = smiles[i:i+self._batch_size]
            enc = _encode_from_smiles(self._dataset, self._model, batch)
            rets.extend(enc)
        return np.stack(rets, axis=0)

    def load_pretrained(self, filename, *args, **kwargs):
        self._dataset, self._model = load_vae_model_and_dataset(filename)
        assert isinstance(self._model, MoLeRVae), self._model
    
    @classmethod
    def from_config(cls, config):
        return cls()

In [33]:
# Regression Transformer implementation

import random


class RegressionTransformer(BaseModel, DistributionMatchingGenerator, SmilesEncoder):
    def __init__(self) -> None:
        super().__init__()
        self._model: RegressionTransformerGeneratorWrapper = None
        self._dataset = None
        self._tokenizer = None
        self._batch_size = None
        self._temperature = None
        self._top_k = None
        self._device = None
        self._tolerence  = None
        self._algorithm_version = None
        self._search_strategy = None 
        self._target = None
        self._fraction_to_mask = None


    def to_guacamole_generator(
            self, tokenizer, batch_size, temperature, top_k, device, tolerence,
            target, search_strategy, fraction_to_mask, algorithm_version) -> DistributionMatchingGenerator:
        self._tokenizer = tokenizer
        self._batch_size = batch_size
        self._temperature = temperature
        self._top_k = top_k
        self._device = device
        self._tolerence  = tolerence 
        self._algorithm_version = algorithm_version
        self._search_strategy =  search_strategy
        self._target = target
        self._fraction_to_mask = fraction_to_mask
        return self
    
    def to_smiles_encoder(self, tokenizer, batch_size, device) -> SmilesEncoder:
        self._tokenizer = tokenizer
        self._batch_size = batch_size
        self._device = device
        return self

    def _generate_single_example(self) -> str:
        assert self._dataset is not None, "Initialize the dataset prior to generation"
        seed_example, seed_property = random.sample(self._dataset, 1)[0]  # get the sampled example

        generator = RegressionTransformerGeneratorWrapper(
            configuration=RegressionTransformerMolecules(
                algorithm_version="qed",
                search="sample",
                temperature=1.0, 
                tolerance=100,
                sampling_wrapper={
                    'property_goal': {'<qed>': seed_property}, 
                    'fraction_to_mask': 0.2
                }),
            target=seed_example)
        generation = generator.sample(1)
        return generation

    def generate(self, number_samples: int):
        generated = []
        with self._model as model:
            for _ in tqdm(range(0, number_samples, self._batch_size), "Generating samples"):
                samples = model.sample(self._batch_size)
                generated.extend(samples)
        return generated[:number_samples]

    def encode(self, smiles: list[str]) -> np.ndarray:
        rets = []
        for i in tqdm(range(0, len(smiles), self._batch_size), "Encoding samples"):
            batch = smiles[i:i+self._batch_size]
            enc = _encode_from_smiles(self._dataset, self._model, batch)
            rets.extend(enc)
        return np.stack(rets, axis=0)

    def load_pretrained(self, filename, *args, **kwargs):
        self._dataset = _load_regression_transformer_dataset(filename)
    
    @classmethod
    def from_config(cls, config):
        return cls()

In [34]:
model = RegressionTransformer()

In [35]:
FILENAME = '/home/aih/gawade.pankhil/.conda/regression-transformer/data/qed/chembl_smiles_eval.txt'
model.load_pretrained(FILENAME)

In [36]:
model._generate_single_example()



INFO:gt4sd.algorithms.core:runnning RegressionTransformer with configuration=RegressionTransformerMolecules(algorithm_version='qed', search='sample', temperature=1.0, batch_size=8, tolerance=100.0, sampling_wrapper={'property_goal': {'<qed>': 'CCCN(CCC)c1c(C)nc(nc1OC)-c1c(OC)cccc1OC'}, 'fraction_to_mask': 0.2})
INFO:gt4sd.algorithms.conditional_generation.regression_transformer.core:ensure artifacts for the application are present.
INFO:gt4sd.s3:starting syncing
INFO:gt4sd.s3:syncing complete
INFO:gt4sd.s3:starting syncing
INFO:gt4sd.s3:syncing complete


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'InferenceBertTokenizer'.
/home/aih/gawade.pankhil/miniconda3/envs/rtgt4sd/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1132: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
/home/aih/gawade.pankhil/miniconda3/envs/rtgt4sd/lib/python3.9/site-packages/transformers/modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct mali

INFO:gt4sd.algorithms.conditional_generation.regression_transformer.implementation:Model restored from /home/aih/gawade.pankhil/.gt4sd/algorithms/conditional_generation/RegressionTransformer/RegressionTransformerMolecules/qed


TypeError: No registered converter was able to produce a C++ rvalue of type std::basic_string<wchar_t, std::char_traits<wchar_t>, std::allocator<wchar_t> > from this Python object of type float

In [17]:
random.sample(model._dataset, 1)[0]

(0.394, 'Nc1ccc(C=CC(=O)c2ccc(I)s2)cc1')

In [ ]:
import logging
import csv
from gt4sd.algorithms.conditional_generation.regression_transformer import (
    RegressionTransformer, RegressionTransformerMolecules
)
from rdkit import Chem
import logging
import os
import pkgutil
import tempfile
import time
from typing import List
import pandas as pd 
import fcd
import numpy as np
import uuid

from guacamol.distribution_learning_benchmark import DistributionLearningBenchmark, DistributionLearningBenchmarkResult
from guacamol.distribution_matching_generator import DistributionMatchingGenerator
from guacamol.utils.data import get_random_subset
from guacamol.utils.sampling_helpers import sample_valid_molecules

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# SMILES string for the target molecule
#smi = "CC(C#C)N(C)C(=O)NC1=CC=C(Cl)C=C1"

smi="CCCN(CCc1cccc(-c2ccccc2)c1)C(=O)C1OC(C(=O)O)=CC(N)C1NC(C)=O"
target_qed = 0.0
target_esol = -3.53

# Configuration for the RegressionTransformerMolecules
config = RegressionTransformerMolecules(
    algorithm_version="qed",
    search="sample",  # The alternative is 'greedy' but 'sample' is recommended for generative tasks
    temperature=1.0, 
    tolerance=100,  # Percentage of tolerated deviation from the desired property value (here -3.53)
    sampling_wrapper={
        'property_goal': {'<qed>': target_qed}, 
        'fraction_to_mask': 0.2
    }
)

# Initialize the RegressionTransformer with the configuration
qed_generator = RegressionTransformer(
    configuration=config, target=smi
)

generations = list(qed_generator.sample(10000))
#generations = list(esol_generator.sample(1000))
# Generate a unique identifier for the file names
unique_id = uuid.uuid4()
# Define the output file path
output_file = f'generations_{unique_id}.csv'
summary_file = f'output_summary_{unique_id}.txt'
#output_file = 'generations_16.csv'

# Write to the CSV file
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    # If 'generations' is a list of strings, use the following
    for generation in generations:
        writer.writerow([generation])

def load_smiles(file_path):
    with open(file_path, 'r') as file:
        smiles_list = [line.strip() for line in file]
    return smiles_list

reference_smiles = load_smiles('./RT/guacamol_v1_train.smiles')

#generations = pd.read_csv('generations.csv')
#generations = [ast.literal_eval(generation) for generation in generations]
generated_molecules = [i for i ,j in generations]
chemnet_model_filename = 'ChemNet_v0.13_pretrained.pt'
sample_size=30000
reference_molecules = get_random_subset(reference_smiles, sample_size, seed=42)

def _load_chemnet():
        """
        Load the ChemNet model from the file specified in the init function.

        This file lives inside a package but to use it, it must always be an actual file.
        The safest way to proceed is therefore:
        1. read the file with pkgutil
        2. save it to a temporary file
        3. load the model from the temporary file
        """
        model_bytes = pkgutil.get_data('fcd', chemnet_model_filename)
        assert model_bytes is not None

        tmpdir = tempfile.gettempdir()
        model_path = os.path.join(tmpdir, chemnet_model_filename)

        with open(model_path, 'wb') as f:
            f.write(model_bytes)

        return fcd.load_ref_model(model_path)
chemnet = _load_chemnet()

def _calculate_distribution_statistics( model, molecules: List[str]):
        sample_std = fcd.canonical_smiles(molecules)
        gen_mol_act = fcd.get_predictions(model, sample_std)

        mu = np.mean(gen_mol_act, axis=0)
        cov = np.cov(gen_mol_act.T)
        return mu, cov
mu_ref, cov_ref =_calculate_distribution_statistics(model=chemnet,molecules=reference_molecules)
mu, cov = _calculate_distribution_statistics(model=chemnet, molecules=generated_molecules)
FCD = fcd.calculate_frechet_distance(mu1=mu_ref, mu2=mu,
                                             sigma1=cov_ref, sigma2=cov)
score = np.exp(-0.2 * FCD)

metadata = {
            'number_reference_molecules': len(reference_molecules),
            'number_generated_molecules': len(generated_molecules),
            'FCD': FCD,
            'score': score

        }
with open(summary_file, 'w') as f:
    f.write("Configuration Details:\n")
    for key, value in config.__dict__.items():
        f.write(f"{key}: {value}\n")

    f.write("\nMetadata:\n")
    for key, value in metadata.items():
        f.write(f"{key}: {value}\n")

print(f'Results have been saved to {summary_file}')